# Assignment 4

*Submission From:*

*18EE10043 : Rhitvik Sinha*  
*18EE10062 : Akshat Kumar Gupta*  
*18EE30021 : Pratyush Jaiswal*  
*18EE30029 : Nuruddin Jiruwala*

*Performed in Google Colab*

In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Part 1

*Read the .mat files in Python and access the variables.*

In [ ]:
import scipy.io
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso

mat=scipy.io.loadmat('/content/drive/My Drive/ML_datasets/2010rainfall.mat')
mat1=scipy.io.loadmat('/content/drive/My Drive/ML_datasets/2011rainfall.mat')

print(mat.keys())
print(mat1.keys())

data = mat['XR1']
data1 = mat1['XR']

np.shape(data1)

dict_keys(['__header__', '__version__', '__globals__', 'XR1', 'ZR1'])
dict_keys(['__header__', '__version__', '__globals__', 'XR', 'ZR'])


(357, 122)

In [ ]:
ind = list(range(1, 358))
cols = list(range(1, 123))

Train_main_df = pd.DataFrame(data, index = ind, columns = cols)   # 2010 training data
Test_main_df1 = pd.DataFrame(data1, index = ind, columns = cols)    # 2011 testing data

Train_main_df.tail()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122
353,15.400000,22.299999,3.6,18.700003,5.200000,34.400005,11.800001,9.9,10.400001,33.800003,18.400002,2.600000,14.600001,9.9,2.7,0.0,2.000000,10.2,12.400001,21.700001,4.400000,13.200001,3.800000,0.0,1.2,22.700003,21.400000,14.200001,0.3,4.000000,26.600002,0.400000,0.0,3.200000,3.200000,6.6,15.200001,24.100000,25.600000,25.200001,...,21.0,6.200000,0.0,0.0,0.0,95.800011,20.500000,1.300000,0.0,13.200001,7.600000,10.5,0.0,2.600000,52.100002,46.000004,35.799999,3.200000,96.400002,77.100006,11.4,0.200000,0.0,10.5,21.100000,83.300003,6.100000,0.0,19.799999,8.000000,0.200000,0.0,0.1,52.299999,0.0,44.599998,28.000002,1.700000,1.700000,0.0
354,10.266666,14.866666,2.4,12.466667,3.466666,22.933334,7.866667,6.6,6.933333,22.533335,12.266667,1.733333,9.733335,6.6,1.8,0.0,1.333333,6.8,8.266666,14.466667,2.933333,8.800000,2.533334,0.0,0.8,15.133334,14.266666,9.466667,0.2,2.666667,17.733335,0.266667,0.0,2.133333,2.133333,4.4,10.133334,16.066668,17.066668,16.800001,...,14.0,4.133333,0.0,0.0,0.0,63.866669,13.666667,0.866667,0.0,8.800000,5.066667,7.0,0.0,1.733333,34.733334,30.666666,23.866667,2.133333,64.266670,51.400005,7.6,0.133333,0.0,7.0,14.066667,55.533337,4.066667,0.0,13.200000,5.333333,0.133333,0.0,0.0,34.866665,0.0,29.733332,18.666666,1.133333,1.133333,0.0
355,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
356,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
357,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0


## Part 2

*Use a linear regression model to predict the rainfall XR(s,t) at any location 's' on day 't', using as predictor the rainfall at all other locations on the same day, and also rainfall at the same location on the previous 2 days* `[XR(1,t)....XR(s-1,t), XR(s+1,t),....XR(357,t), XR(s,t-1), XR(s,t-2)]`*. Use 2010 data for training.*

*Build such a model for* `s=42` *(Mumbai),* `s=158` *(Delhi),* `s= 299` *(Kharagpur).*

In [ ]:
def Trainable(Train,s):
    copyt = Train.T             # transpose as working with columns is more suitable/easy
    rainlist = copyt[s].tolist()        # rain at place s for 122 days
    rainone = [0.0]                     # day -1 assumed to be zero
    raintwo = [0.0, 0.0]                # day -1 and -2 assumed to be zero
    # print(rainlist)
    rainone = rainone + rainlist
    raintwo = raintwo + rainlist
    rainone = rainone[:-1]
    raintwo = raintwo[:-2]              # getting t-1 and t-2 data and assuming zero rain at day -1 and -2
    Y_train = copyt[s]
    X_train = copyt.drop(s, axis=1)     # all 122 days data except place s
    X_train['t-1'] = rainone            # adding two cols for the previous days rain at place s
    X_train['t-2'] = raintwo
    return X_train,Y_train

In [ ]:
def Linear(s):
    X_train,Y_train=Trainable(Train_main_df,s)
    print(X_train.shape)
    print(Y_train.shape)
    linreg = LinearRegression()
    linreg.fit(X_train, Y_train)
    return linreg

In [ ]:
Bombay = Linear(42)
Delhi = Linear(158)
Kharagpur = Linear(299)                 #training for three cities

(122, 358)
(122,)
(122, 358)
(122,)
(122, 358)
(122,)


In [ ]:
Test_main_df1.head()                    # 2011 dataset

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122
1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.185278,0.0,0.0,0.000000,0.000000,2.033708,0.000000,0.184883,0.000000,2.680797,4.948352,5.173365,0.138860,...,0.369765,0.000000,0.0,0.0,0.381641,1.078712,2.762939,15.534396,31.112030,19.460709,5.486663,5.964305,17.108110,3.543520,2.559318,127.579361,121.933098,3.120047,0.0,0.000000,15.017424,8.859507,19.032047,0.000000,0.000000,2.320731,3.249167,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.958035,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,2.008009,11.046118,23.083147,18.015162,11.933199,12.927632,34.816326,0.000000,0.000000,72.575233,78.449051,5.970378,0.0,0.000000,10.052490,2.070224,29.887758,0.000000,0.000000,0.000000,2.012872,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,21.500555,0.0,0.0,0.0,0.0,0.0,0.0,1.325026,4.442735,7.948280,75.593704,8.080275,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,9.776277,5.391882,2.522401,7.840161,12.483876,17.858332,19.607910,...,2.305163,35.359970,0.0,0.0,10.266807,7.287509,29.613338,5.146783,38.125687,0.545599,1.091198,6.700237,6.558243,22.360601,85.824715,186.834686,75.342026,4.566547,0.0,0.000000,0.000000,30.210543,15.056442,0.000000,0.000000,2.149278,38.689827,0.0,0.0,0.0,0.0,0.0,0.0,1.563111,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.639077,6.604940,0.108992,0.0,0.0,0.0,0.0,0.0,0.217983,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.290645,0.326975,0.000000,9.373127,1.250425,0.326975,0.000000,34.397381,11.856451,15.145469,...,0.000000,16.189949,0.0,0.0,3.344543,3.935243,8.378140,33.333427,54.624939,13.371160,0.363306,19.586903,0.625213,22.111752,66.274300,138.969025,59.655888,2.638884,0.0,0.000000,0.494259,48.747658,7.988505,2.216165,1.036551,4.511424,1.015970,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.232115,0.000000,0.000000,0.000000,0.541212,6.702410,6.658057,1.495367,...,0.000000,0.000000,0.0,0.0,0.000000,0.121333,1.944354,16.418554,45.220749,8.702755,0.470484,4.417555,12.431360,3.968278,13.383103,39.888123,210.623093,2.181559,0.0,1.764315,1.192227,2.661123,5.596737,0.000000,0.676007,1.088342,4.301603,0.0,0.0,0.0,0.0,0.0,0.0,0.588105,0.0,0.0,0.0,0.0,0.0,0.0


### Predicting on test data

In [ ]:
def Predictmodel(model, s):             # model is the trained model and s is the city number
    X_test,Y_test=Trainable(Test_main_df1,s)
    Y_pred = model.predict(X_test)      # predicting on 2011
    Y_pred = [0.0 if x<0 else x for x in Y_pred]# amount of rain cannot be negative, so replace by zero
    return Y_pred

In [ ]:
Y_bombay = Predictmodel(Bombay, 42)
Y_delhi = Predictmodel(Delhi, 158)
Y_kgp = Predictmodel(Kharagpur, 299)
      

Y_bombay_actual = Test_main_df1.T[42]
Y_delhi_actual = Test_main_df1.T[158]
Y_kgp_actual = Test_main_df1.T[299]

## Part 3

*Use the same model to predict the rainfall at these 3 locations on each day of 2011.  Use values in XR as predictors. Compare the results with the true values and compute error for 3 locations separately.*

In [ ]:
from sklearn.metrics import mean_squared_error

rmsbombay = (mean_squared_error(Y_bombay_actual, Y_bombay))
rmsdelhi = (mean_squared_error(Y_delhi_actual, Y_delhi))
rmskgp = (mean_squared_error(Y_kgp_actual, Y_kgp))
print("Bombay error:" + str(rmsbombay))
print("Delhi error:" + str(rmsdelhi))
print("Kgp error:" + str(rmskgp))

Bombay error:1204.7749014858373
Delhi error:448.46136659206354
Kgp error:419.35458442698535


## Part 4

*Repeat the same process using LASSO linear regression. Using the coefficients, identify the top 5 predictors for each of the 3 locations.*

In [ ]:
def Lassoreg(s,p):
    X_train,Y_train=Trainable(Train_main_df,s)
    model = Lasso(alpha=p, tol=0.01)       # training on lasso
    model.fit(X_train, Y_train)
    # print('abc')
    return model, X_train

In [ ]:
lassoB, _ = Lassoreg(42, 40)
print(np.count_nonzero(lassoB.coef_))
lassoD, _ = Lassoreg(158, 60)
print(np.count_nonzero(lassoD.coef_))         # getting top five coefficients by changing alpha
lassoK, _ = Lassoreg(299, 60)
print(np.count_nonzero(lassoK.coef_))

5
5
5


In [ ]:
indexB=[]
for x in lassoB.coef_:
  if x!=0:
    indexB.append(list(lassoB.coef_).index(x))


indexD=[]
for x in lassoD.coef_:
  if x!=0:
    indexD.append(list(lassoD.coef_).index(x))          # getting index for non zero values

indexK=[]
for x in lassoK.coef_:
  if x!=0:
    indexK.append(list(lassoK.coef_).index(x))

In [ ]:
print(indexB)
print(indexD)
print(indexK)           #The features at these column numbers are most important out of 358

[25, 39, 40, 314, 315]
[25, 132, 158, 160, 214]
[161, 230, 232, 247, 303]


In [ ]:
y_pred_lasso_B=Predictmodel(lassoB,42)
y_pred_lasso_D=Predictmodel(lassoD,158)
y_pred_lasso_K=Predictmodel(lassoK,299)

In [ ]:
rmsbombay = (mean_squared_error(Y_bombay_actual, y_pred_lasso_B))
rmsdelhi = (mean_squared_error(Y_delhi_actual, y_pred_lasso_D))
rmskgp = (mean_squared_error(Y_kgp_actual, y_pred_lasso_K))
print("Bombay error:" + str(rmsbombay))
print("Delhi error:" + str(rmsdelhi))
print("Kgp error:" + str(rmskgp))

Bombay error:1029.5747422659927
Delhi error:177.65386470515926
Kgp error:406.0173392249512


## Part 5

*Use Decision Tree on 2010 data to classify each day as 1 or 0 (as given in ZR1). For each day, use the 357-dimensional rainfall vector as feature vector. Report the 10 most discriminative features (i.e. locations)*

In [ ]:
train_vector = mat['ZR1'][0]
test_vector=mat1['ZR'][0]
print(np.shape(Train_main_df))
print(np.shape(train_vector))       #checking the dimensions and getting the vectors ready

(357, 122)
(122,)


In [ ]:
from sklearn.tree import DecisionTreeClassifier
treem=DecisionTreeClassifier(criterion='entropy',random_state=5)
treem.fit(Train_main_df.T,train_vector.T)  # the input shapes must be (122,357) and (122,)  i.e. (num_labels,num_features) for X and (num_labels) for Y

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=5, splitter='best')

In [ ]:
all_feat=treem.feature_importances_

In [ ]:
np.count_nonzero(all_feat)  # how many features have non zero weight/importance

8

In [ ]:
imp_feat=np.nonzero(all_feat)[0]

In [ ]:
imp_feat # col numbers of 8 most important features

array([ 29,  87, 123, 159, 216, 255, 314, 337])

## Part 6

*Use this Decision Tree to classify each day of 2011 as 1 or 0. Report accuracy by comparing with ZR.*

In [ ]:
prediction_tree=treem.predict(Test_main_df1.T)  # predicting on 2011 data

In [ ]:
prediction_tree

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1,
       0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)

In [ ]:
test_vector

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)

In [ ]:
count=0
for i in range(len(test_vector)):
  if prediction_tree[i]==test_vector[i]:          # checking how many values match
    count=count+1

In [ ]:
accuracy=count*100/len(test_vector)
print("Accuracy : ",accuracy," %")      #ACCURACY

Efficiency :  77.04918032786885  %
